In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_r50_fpn_1x_coco_swin_L.py')

root='../../dataset/'

# dataset config 수정
# cfg.data.train.classes = classes
# cfg.data.train.img_prefix = root
# cfg.data.train.ann_file = root + 'train_MultiStfKFold.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize

cfg.data.train.dataset.classes = classes
cfg.data.train.dataset.img_prefix = root
cfg.data.train.dataset.ann_file = root + '5___train_MultiStfKFold.json' # train json 정보
#cfg.data.train.dataset.ann_file = root + 'train_MultiStfKFold.json' # train json 정보
#cfg.data.train.pipeline[0]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + '5___val_MultiStfKFold.json' # train json 정보
cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize


cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.samples_per_gpu = 2
#cfg.resume_from = '../../trash/epoch_36.pth'

import datetime
now = (datetime.datetime.now().replace(microsecond=0) + datetime.timedelta(hours=9)).strftime("%m-%d %H:%M")


cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/cascade_rcnn_r50_fpn_1x_coco_swin_L_{now}'

# cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.optimizer.type = "Adam"
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)
cfg.evaluation = dict(save_best='bbox_mAP_50',metric='bbox')
cfg.device = get_device()

In [6]:
# cfg.log_config.hooks = [dict(type='TextLoggerHook'),
#                         dict(type='WandbLoggerHook',
#                              init_kwargs=dict(project='project_0'))
#                        ]
cfg.optimizer

{'type': 'AdamW', 'lr': 0.0001}

In [7]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [8]:
# dataset 확인
datasets[0]

In [9]:

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={
            'project': 'trash_detection_swin',
            'entity': 'doyunkim',
            'name' : f'{cfg.model.type}_{now}_f',
            'tags': [cfg.model.backbone.type, cfg.model.neck.type],
            # 'tags': [cfg.model.backbone.type, cfg.model.neck.type, str(cfg.data.train.pipeline[2]['img_scale'][0])]
            },
         interval=10,
         # log_checkpoint=False,
         # log_checkpoint_metadata=False,
         # num_eval_images=10,
         # bbox_score_thr=0.3
        )]
         

In [10]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-11-30 08:19:48,374 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22k.pth
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22k.pth" to /opt/ml/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22k.pth


  0%|          | 0.00/874M [00:00<?, ?B/s]

2022-11-30 08:23:19,284 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-11-30 08:23:19,309 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-11-30 08:23:19,319 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-11-30 08:23:19,423 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls_fcs'}, {'name': 'reg_fcs'}]}]
2022-11-30 08:23:19,527 - mmdet - INFO - initialize Sh

In [11]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-11-30 08:23:22,884 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-11-30 08:23:23,473 - mmdet - INFO - Start running, host: root@a78b80fabad9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/cascade_rcnn_r50_fpn_1x_coco_swin_L_11-30 17:19
2022-11-30 08:23:23,475 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -----------------

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/runner/hooks/evaluation.py:226: UserWarning: runner.meta is None. Creating an empty one.
  warnings.warn('runner.meta is None. Creating an empty one.')
wandb: Currently logged in as: doyunkim. Use `wandb login --relogin` to force relogin


2022-11-30 08:24:25,977 - mmdet - INFO - Epoch [1][50/1953]	lr: 1.441e-05, eta: 7:09:14, time: 1.101, data_time: 0.052, memory: 20750, loss_rpn_cls: 0.5534, loss_rpn_bbox: 0.0859, s0.loss_cls: 0.5561, s0.acc: 90.9316, s0.loss_bbox: 0.1055, s1.loss_cls: 0.2235, s1.acc: 92.5586, s1.loss_bbox: 0.0332, s2.loss_cls: 0.1040, s2.acc: 93.0742, s2.loss_bbox: 0.0053, loss: 1.6668, grad_norm: 10.5672
2022-11-30 08:25:19,145 - mmdet - INFO - Epoch [1][100/1953]	lr: 1.891e-05, eta: 7:00:56, time: 1.063, data_time: 0.005, memory: 20750, loss_rpn_cls: 0.2759, loss_rpn_bbox: 0.0590, s0.loss_cls: 0.3693, s0.acc: 92.1660, s0.loss_bbox: 0.1785, s1.loss_cls: 0.1012, s1.acc: 96.3809, s1.loss_bbox: 0.0517, s2.loss_cls: 0.0311, s2.acc: 97.9746, s2.loss_bbox: 0.0072, loss: 1.0738, grad_norm: 4.7658
2022-11-30 08:26:06,839 - mmdet - INFO - Epoch [1][150/1953]	lr: 2.341e-05, eta: 6:43:25, time: 0.954, data_time: 0.005, memory: 20750, loss_rpn_cls: 0.1909, loss_rpn_bbox: 0.0683, s0.loss_cls: 0.4171, s0.acc: 90.1

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 977/977, 2.2 task/s, elapsed: 435s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.20s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.66s).
Accumulating evaluation results...


2022-11-30 16:36:41,318 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.644
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.520
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.135
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.577
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.099
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.279
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=0.58s).


2022-11-30 16:36:41,746 - mmdet - INFO - The previous best checkpoint /opt/ml/baseline/mmdetection/work_dirs/cascade_rcnn_r50_fpn_1x_coco_swin_L_11-30 17:19/best_bbox_mAP_50_epoch_10.pth was removed
2022-11-30 16:36:49,540 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_50_epoch_12.pth.
2022-11-30 16:36:49,542 - mmdet - INFO - Best bbox_mAP_50 is 0.6440 at 12 epoch.
2022-11-30 16:36:49,543 - mmdet - INFO - Epoch(val) [12][977]	bbox_mAP: 0.4930, bbox_mAP_50: 0.6440, bbox_mAP_75: 0.5200, bbox_mAP_s: 0.0160, bbox_mAP_m: 0.1350, bbox_mAP_l: 0.5770, bbox_mAP_copypaste: 0.493 0.644 0.520 0.016 0.135 0.577


learning_rate,▄▆████████▇▇▇▆▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,▃▃▅▅█▄▅▃▄▆▆▃▆▃▆▂▃▅▄▅▄▃▃▄▁▄▃▄▄▁▂▂▃▂▃▃▂▁▂▇
train/loss,▇▄██▅▃▄▃▄▅▃▄▄▃▅▂▃▃▄▂▂▂▃▂▂▃▂▃▂▁▃▂▂▂▂▂▁▁▂▂
train/loss_rpn_bbox,█▂▆▇▄▂▄▃▅▆▂▃▃▂▅▂▃▃▃▂▂▁▃▂▄▅▁▃▂▁▂▂▃▃▂▁▂▂▂▂
train/loss_rpn_cls,█▃▆▅▄▃▂▂▅█▂▂▂▂▃▂▃▂▂▂▁▂▂▂▃▃▂▃▂▁▂▂▂▂▂▂▂▁▁▁
train/s0.acc,▂▆▁▂▃▆▅▅▅▄▆▅▅▆▅▇▅▆▅▇▇▇▆▇▇▆▇▆▇█▆▇▇▇█▇█▇▇▇
train/s0.loss_bbox,█▃▇▅▄▂▂▃▄▃▂▃▃▃▄▁▃▃▃▂▂▂▃▂▃▃▂▂▂▁▂▂▂▂▂▂▁▁▂▂
train/s0.loss_cls,█▄██▅▃▄▃▄▅▃▄▄▃▄▂▃▃▄▂▂▂▃▂▂▂▂▂▂▁▃▁▂▂▁▂▁▁▂▁
train/s1.acc,▄▅▁▁▂▅▄▅▅▄▅▅▅▆▄▇▄▆▄▆▇▇▆▇▇▆▇▇▆█▆█▇▇█▇▇█▇▇
train/s1.loss_bbox,▄▄█▆▆▃▃▃▄▃▃▃▃▃▄▂▃▄▃▃▂▂▃▂▂▄▂▂▂▁▃▂▃▃▂▃▁▁▃▂
